#Import libraries

In [0]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString, MultiPolygon
import os
import glob
from shapely import geometry
import numpy as np
import time
import shutil

#Import cloud files

In [0]:
inPath = "/dbfs/mnt/strukturparametre/treecrownblocks/"  # input tree crown or classification polygons folder path
gpkg_pattern = os.path.join(inPath, '*.gpkg')
treepaths = glob.glob(gpkg_pattern)
gdf_nofor = gpd.read_file("/dbfs/mnt/strukturparametre/Non_forest_eraser_repaired.shp")  #input non forest layer
gdf_nofor = gpd.GeoDataFrame(gdf_nofor, crs="EPSG:25832")


In [0]:
treepaths

Out[3]: ['/dbfs/mnt/strukturparametre/planet_forest/forest_planet.gpkg']

#Define tree-id function

In [0]:
def calc_trees_id(filename):
    filep = filename
    filen = os.path.splitext(os.path.basename(filename))[0] 
    print('reading', filen)
    treecrowns = gpd.read_file(filep)
    
    gdf_trees = gpd.GeoDataFrame(treecrowns, crs="EPSG:25832")    


    #Processing grid creation for chunks       

    minX, minY, maxX, maxY = gdf_trees.total_bounds

    # Create a grid to the extent of input tree crowns
    x, y = (minX, minY)
    geom_array = []

    # Polygon Size
    square_size = 1000
    while y <= maxY:
        while x <= maxX:
            geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
            geom_array.append(geom)
            x += square_size
        x = minX
        y += square_size

    gdf_grid = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:25832')

    gdf_trees = gdf_trees.assign(trees='tree')

    gdf_trees =gdf_trees.to_crs(epsg=25832)



    #extract by location so that processing grid is only where there are tree crown polygons



    #field calcluator to number row for new grid id

    gdf_grid['row'] = np.arange(len(gdf_grid))


    # create spatial join trees with grid for grid id that will determine processing chunks

    dfsjoin = gpd.sjoin(gdf_trees, gdf_grid) #Spatial join Points to polygons

    dfpivot = pd.pivot_table(dfsjoin,index='row',columns='trees',aggfunc={'trees':len})  

    dfpivot.columns = dfpivot.columns.droplevel()  


    dfpolynew = gdf_grid.merge(dfpivot, how='left', on='row')

    grid_with_trees = dfpolynew.dropna( how='any', subset=['tree'])

    grid_with_trees['grid_id'] = np.arange(len(grid_with_trees))

    trees_id = gpd.sjoin(gdf_trees, grid_with_trees)

    trees_id = trees_id[["geometry", "grid_id"]]
    print('trees with processing grid created for', filen)

    # #count features of grid gpkg
    return grid_with_trees["grid_id"].to_list(), trees_id

#Define chunk-related functions

In [0]:
def buffer_in_correction(bfr_in_input):
    
    bfr_gdf = bfr_in_input.explode(index_parts=True)
    bfr_gdf = gpd.GeoDataFrame(gpd.GeoSeries(bfr_gdf))
    
    bfr_gdf = bfr_gdf.rename(columns={0:'geometry'}).set_geometry('geometry')
        
    
    bfr_gdf = bfr_gdf.set_crs(25832)
    return bfr_gdf
def buffer_out_correction(bfr_out_output):
    bfr_gdf = gpd.geoseries.GeoSeries([geom for geom in bfr_out_output.unary_union.geoms])
    bfr_gdf = bfr_gdf.set_crs(25832)
    return bfr_gdf

#creation of chunks from selection by grid id iteration and then buffer out 
def conc_chunks(i, trees_id):
    #extract by attribute
    chunk = gpd.GeoDataFrame(trees_id[trees_id['grid_id'] == i])
    #buffer out 10 m
    chunk = chunk.to_crs(25832).buffer(10)
    chunk = gpd.GeoDataFrame(gpd.GeoSeries(chunk))
    chunk=  chunk.rename(columns={0:'geometry'}).set_geometry('geometry')
 #   chunk = gpd.geoseries.GeoSeries([geom for geom in chunk.unary_union.geoms])
    chunk = gpd.geoseries.GeoSeries(chunk["geometry"].unary_union)
    chunk = chunk.set_crs('epsg:25832')
    return chunk

# Define deliniation

In [0]:
def delineation_process(filename, list_chunks):
    #acquiring filename from filepath
    filen = os.path.splitext(os.path.basename(filename))[0]
    

    st= time.time()

 
    print('merging chunks',filen)
    
    #merge chunks
    merge = pd.concat(list_chunks)
    
    merge = gpd.GeoDataFrame(gpd.GeoSeries(merge))
    
    merge = merge.rename(columns={0:'geometry'}).set_geometry('geometry')
    
    #getting extent of merged geodataframe in order for the non forest to be clipped in that extent
    
    extent= merge.total_bounds
    [xmin, ymin, xmax, ymax] = merge.total_bounds
    
    p1 = geometry.Point(xmin,ymin)
    p2 = geometry.Point(xmax,ymin)
    p3 = geometry.Point(xmax,ymax)
    p4 = geometry.Point(xmin,ymax)

    pointList = [p1, p2, p3, p4]

    poly = geometry.Polygon(pointList)

    extent = gpd.GeoSeries.from_wkt([poly.wkt])

    mask_gpd = extent.set_crs("EPSG:25832")


    nonfor_clip = gdf_nofor.clip(mask_gpd, keep_geom_type=True)
    
        
    merge = merge.to_crs(25832).buffer(-20)
    

    merge = buffer_in_correction(merge)
    

    print('1st buffer 20m in for',filen)



    # Buffer out 20 3rd
    merge = merge.to_crs(25832).buffer(20)
    merge = buffer_out_correction(merge)
 
    # Buffer_in_10_2 4th
    merge = merge.to_crs(25832).buffer(-10)
    
    merge = buffer_in_correction(merge)
    
    #Buffer out 10 5th
    
    merge = merge.to_crs(25832).buffer(10)
    merge = buffer_out_correction(merge)

    # Buffer_in_10_3 6th

    merge = merge.to_crs(25832).buffer(-10)
    merge = buffer_in_correction(merge)
    
    
            # Difference_final
    merge = merge.overlay(nonfor_clip, how='difference', make_valid=True)
    
    print('difference_final')

        # Buffer_in_10_4 8th
    merge = merge.to_crs(25832).buffer(-10)
    
    merge = buffer_in_correction(merge)
    

    # Buffer_out_10_3 9nth
    merge = merge.to_crs(25832).buffer(10)
    
    merge = gpd.geoseries.GeoSeries([geom for geom in merge.unary_union.geoms])
    
    print('fix union')
    
    merge = merge.set_crs('epsg:25832')


    
    
    merge = merge.explode(index_parts=True)

    print('explode')

    merge = gpd.GeoDataFrame(gpd.GeoSeries(merge))
    
    merge = merge.rename(columns={0:'geometry'}).set_geometry('geometry')
    # Field calculator
    merge['area_ha'] = merge.area/10000
    

    # Extract by attribute
    forest = merge[merge['area_ha'] >= 0.5]
    
    #output of delineated forest block
    
    forest.to_file(f"/tmp/forest_{filen}.gpkg", layer=f'forest_{filen}', driver="GPKG")
    shutil.move(f"/tmp/forest_{filen}.gpkg", f"/dbfs/mnt/strukturparametre/forestblocks/forest_{filen}.gpkg" )
    
    et = time.time()
    total_time = et - st
    print(total_time, 's , elapsed_time')
    return forest

# Combine functions

In [0]:
def from_filename_to_delineation(filename):
    grid_w_trees_list,trees_id_pd = calc_trees_id(filename)
    chunk_list = []
    for i in range(len(grid_w_trees_list)): 
        chunk_part = conc_chunks(i, trees_id_pd)
        chunk_list.append(chunk_part)
    delineation_process(filename, chunk_list)

##Paralellize deliniation per cloud-file

In [0]:
file_rdd = sc.parallelize(treepaths, len(treepaths))
write_files_rdd = file_rdd.map(lambda x : from_filename_to_delineation(x))
write_files_rdd.collect()